# Background

State machines, like most transition systems, are defined in terms of their set of states, their initial state, their terminal states, and the valid transitions between the states. State machines are still used widely in game character AI, but they also show up in various guises all over computer science (for example in regular expressions, program verification, cyber-physical systems, ...). A solid understanding of transition systems is key to understanding many different research areas.

Today's assignment is to use state machines in a couple of different ways to get used to reading and writing Python code, and to get a feel for what can be done with state machines.

First, we want to create a data type to represent state machines. In Python, the custom data type is the `class`. We know that it will be initialized with a set of states and edges, an initial state, and some terminal states, and creating a state machine should look something like:

```python
ab_a_ = StateMachine(
    #We can define the states and edges in one go
    [("s1", "a", "s2"), ("s2", "b", "s3"), ("s3", "a", "s3")],
    #Initial state
    "s1",
    #Terminal states
    ["s3"]
)
```

To recap:

* `ab_a_ = ...` assigns the result of evaluating the right hand side to the variable `ab_a_`.
* `StateMachine(...)` creates a new object of the StateMachine class
* `[...]` is a comma-separated list of arbitrary "things". This list could grow or shrink but the key idea is that its length is unknown in advance to the code that's processing it (usually necessitating some kind of iteration or recursion over the length of the list).
* `(...)` without a word to its left is a "tuple", a fixed-length sequence of objects. Here, each 3-tuple defines the source state, the transition symbol, and the target state. (If there were a word to the left of the parentheses, it would be a function call as in the case of `StateMachine(...)`).
* `"..."` is a "string", a sequence of characters.

Putting it all together, the line above makes a machine with three states that accepts `aba+`, if written as a regular expression (and stores the machine into the variable `ab_a_`): any string with the prefix `ab` followed by one or more `a`. Here's `ab(cab)+d`, `ab` followed by one or more repetitions of `cab` ending with a `d`:

```python
ab_cab_d = StateMachine(
    [("s1", "a", "s2"),
     ("s2", "b", "s3"), # The same so far...
     ("s3", "c", "s4"),
     ("s4", "a", "s5"),
     ("s5", "b", "s6"),
     ("s6", "c", "s4"), # Note the loop back to s4 on "c"
     ("s6", "d", "s7")], # Or else the continuation to s7 on "d"
    "s1",
    ["s7"]
)
```

We have two questions of interest for any transition system:

* What is the current configuration?
* What configurations are accessible from the current configuration, and how?

In our case, we can use e.g. `ab_a_.state` to get its current state identifier, and `ab_a_.out_edges()` to get a Python `dict` of the successors (with `ab_a_.out_edges().keys()` yielding the symbols and the `ab_a_.out_edgdes().values()` giving the successor states).

There is one operation on state machines that we're interested in right now:

* Advance the state machine along a particular available edge.

We can write e.g. `ab_a_.advance("a")` to advance the `ab_a_` machine by feeding it the symbol `"a"`. If it starts in state `"s1"`, this will put it into state `"s2"`. Then we can call `ab_a_.advance("b")` to feed it a `"b"` and advance it to `"s3"`, and so on. It is an error to call `advance` with an unavailable or invalid symbol.

Now that we know what the StateMachine class should look like, we can define it. Recall that Python is indentation-sensitive:

In [83]:
class StateMachine:
    # __init__ defines the function to be called when `StateMachine(...)` is invoked.
    # Note that, like other instance methods on `StateMachine`, the first argument is `self`:
    # the particular state machine being operated upon (in this case, initialized). `self` is
    # provided implicitly in most cases.
    # `edges` should be a list of 3-tuples of strings, init-state should be a string, and terminals a list of strings.
    def __init__(self, edges, init_state, terminals):
        # We create a `states` variable on `self` to store the transition system information.
        # It will be a dict whose keys are state identifiers and whose values are themselves dicts
        # of successor states keyed by symbols.
        self.states = dict()
        # `for X in C` iterates over each member `X` of collection `C`.
        # Since we know `C` (`edges`) contains 3-tuples, we can "unpack" them using the tuple notation in place of `X`.
        for (src, symbol, dest) in edges:
            # Ensure that the state referred to by `src` exists in the transition system as a dict (again, symbols->states):
            if src not in self.states:
                self.states[src] = dict()
            # Ditto for `dest`.
            if dest not in self.states:
                self.states[dest] = dict()
            # Ensure that the symbol is not already used in an out-edge of src. This is a deterministic state machine which
            # can't make "guesses" in such cases.
            if symbol in self.states[src]:
                # Exceptions are informative failures that other code can catch and deal with; they represent exceptional
                # cases which this code is not in a position to recover from.
                raise Exception("This StateMachine only supports DFAs, so each state can have at most one out edge for each symbol.")
            # Finally, connect the `s`ou`rc`e state to the `dest`ination state along `symbol`.
            self.states[src][symbol] = dest
        # This state machine starts in the initial state
        self.state = init_state
        # And we remember the terminal states so that we know when we're accepting.
        # The list is packed into a `set()`, which permits us to say `state_id in self.terminal_states` to simplify checking.
        # You can think of sets as being like dicts of type Anything->True: 
        # The object is in the set if and only if the key is present.
        self.terminal_states = set(terminals)

    # Determining the available out-edges is straightforward because of the way we've chosen to represent the
    # transition system.
    def out_edges(self):
        # Return the out edges of the currently active state.
        return self.states[self.state]
    
    # A state machine is in a terminal state if its current state is terminal
    def is_terminal(self):
        # Is the current state in the set of terminal states?
        return self.state in self.terminal_states

    # A state machine is stuck if it has no out edges.
    def is_stuck(self):
        return len(self.out_edges()) == 0
    
    # Finally, to advance the state machine by a symbol...
    def advance(self, symbol):
        # We reassign the machine's state to be the target state denoted by that symbol, among the current out edges.
        self.state = self.out_edges()[symbol]

With all that done, we can finally say:

In [84]:
ab_a_ = StateMachine(
    #We can define the states and edges in one go
    [("s1", "a", "s2"), ("s2", "b", "s3"), ("s3", "a", "s3")],
    #Initial state
    "s1",
    #Terminal states
    ["s3"]
)
print("Initial state:", ab_a_.state) # note, no parens after `state` because it's a value field and not a function
print("Accepting?", ab_a_.is_terminal()) # parens after `is_terminal` to _call_ it.
ab_a_.advance("a")
print("After a:", ab_a_.state)
ab_a_.advance("b")
print("After ab:", ab_a_.state)
print("Next steps:", ab_a_.out_edges())
print("Accepting?", ab_a_.is_terminal())

Initial state: s1
Accepting? False
After a: s2
After ab: s3
Next steps: {'a': 's3'}
Accepting? True


# Assignment 1

The first part of the assignment is to define the function `check(...)` below:

In [85]:
def check(sm, string):
    for i in string:
        if i not in sm.out_edges():
            return False
        else:
            sm.advance(i)          
    return sm.is_terminal()

It should give the correct results for at least these examples, and we encourage you to try out more tests.

In [86]:
# Test suite 1. Should not throw any errors.
# (First, we define a function to produce the test state machine so we can use a fresh one every time.)
def test1():
    # This one accepts "h(e(llo|y) | i(hi)*)
    return StateMachine(
        [("s1", "h", "s2"), ("s2", "e", "s3"), 
         ("s3", "y", "s4"), 
         ("s3", "l", "s5"), ("s5", "l", "s6"), ("s6", "o", "s7"),
         ("s2", "i", "s8"), ("s8", "h", "s9"), ("s9", "i", "s8")
        ],
        "s1",
        ["s4", "s7", "s8"]
    )
assert check(test1(), "hello")
assert check(test1(), "hey")
assert check(test1(), "hi")
assert check(test1(), "hihi")
assert check(test1(), "hihihi")
assert not check(test1(), "greetings")
assert not check(test1(), "hel")
assert not check(test1(), "helloy")
assert not check(test1(), "hih")
assert not check(test1(), "heyhey")

Next, define a state machine which accepts email addresses consisting only of the letter "x" and the symbols "@" and "." (in other words, you don't need a ton of edges). Devise tests for it below (try to be sinister---get a partner to come up with adversarial examples too).

In [87]:
def test2():
    # TODO: define transition system here:
    return StateMachine(
        [
            ("s1","x","s2"),
            ("s2","x","s2"),
            ("s2","@","s3"),
            ("s3","x","s4"),
            ("s4","x","s4"),
            ("s4",".","s5"),
            ("s5","x","s6"),
            ("s6","x","s6"),
            ("s6",".","s5"),
        ],
        "s1",
        ["s6"])
# TODO: tests go here
assert check(test2(), "x@x.x")
assert check(test2(), "xxx@x.x")
assert check(test2(), "x@xxx.x")
assert check(test2(), "x@x.xxx")
assert check(test2(), "xxx@xxx.x")
assert check(test2(), "xxx@xxx.xxxx")
assert check(test2(), "xx@x.xxxx")
assert check(test2(), "xx@xxxxxxx.xx")
assert check(test2(), "xx@xxxx.xxx.xx")
assert check(test2(), "xx@xx.x.x.x.xxx.xx.xxxxx.xxx.xx")
assert not check(test2(), "@x.xx")
assert not check(test2(), "xx@.xx")
assert not check(test2(), "xx@xxxx.xxx.")
assert not check(test2(), "xx@.")
assert not check(test2(), "@.")
assert not check(test2(), "@.x")
assert not check(test2(), "xx@xx.x.x..x")
assert not check(test2(), "xx@@xxx..x")
assert not check(test2(), "xxxx@x..x")
assert not check(test2(), "xxx@@xx.xx")

# Assignment 2

Define a function which, given a state machine, generates all the strings of a given (exact) length that the state machine would accept.

If you have an intuition on how to do this, feel free to try it! Insert your attempt and its tests in the empty code cell just after this one. Then come back and take a look at the approach below.

In [88]:
#I couldn't figure out how to do this

def try_sample(sm, length):
    if length == 0:
        return []
    strings = []
    string = ""
    for i in range(0, length):
        if sm.is_terminal:
            strings.append(string)
        else:
            for i in sm.out_edges():
                string += i            
        

Note that any path through a state machine ending in a terminal state is a valid string, and each step of the path is associated with a symbol (i.e., a letter). To combine symbols together into strings, you can use string concatenation:

In [89]:
ab = "a" + "b"
print(ab)
abc = ab + "c"
print(abc)

ab
abc


You can combine lists the same way:

In [90]:
lst_abc = ["a","b"] + ["c"]
print(lst_abc)

['a', 'b', 'c']


All that's left is enumerating the valid paths of a given length and assembling those into strings. The skeleton below is a _recursive_ function that calls itself repeatedly, branching out according to each option. You can imagine that the call to `sample()` is the root of a tree of calls to `sample2`, and the paths through that tree are paths through the state machine---the path itself is "stored" in the third argument to `sample2`. In other words:

* To get the possible paths of length `K` from a state machine `SM`:
    * Initialize `S` to `SM`'s current state.
    * Initialize the `result` to `[]`
    * For each available out-edge at `S`:
        * Advance `SM` along that edge
        * Get the possible paths of length `K-1` from `SM` and add them to the result
        * Put `SM` back into state `S` (so we can advance it along the next available out edge)
    * Yield `result`

It's up to you to figure out how to find the possible paths of length 0. Hint: How many such paths can there be for a given call to `sample2(sm, 0, sofar)`? What aspects of the state machine's status does it depend on?

In [91]:
# Starter function that calls sample2() with an empty string as argument. This is just to "accumulate" paths into
# that argument without tracking a bunch of data externally.
def sample(sm, length):
    return sample2(sm, length, "")

# sample2 samples paths of length "length" from the state machine sm as of its current state.
# sofar is the path assembled so far.
# sample2 returns a list of paths.
def sample2(sm, length, sofar):
    if length == 0:
        if sm.is_terminal():
            return [sofar]
        # TODO: What path(s) should be returned? When?
        else:
            return []
    state = sm.state
    result = []
    for i in sm.out_edges():
        sm.advance(i)
        result += sample2(sm, length-1, sofar + i)
        sm.state = state
    # TODO: implement the rest of the pseudocode above
    return result

In [93]:
assert len(sample(test2(), 0)) == 0
assert len(sample(test2(), 1)) == 0
assert len(sample(test2(), 2)) == 0
assert len(sample(test2(), 3)) == 0
assert len(sample(test2(), 4)) == 0
assert len(sample(test2(), 5)) == 1
assert len(sample(test2(), 6)) == 3
assert len(sample(test2(), 7)) == 7
assert len(sample(test2(), 8)) == 14

all_samples = []
for i in range(0, 9):
    all_samples = all_samples + sample(test2(), i)
print(all_samples)

['x@x.x', 'x@x.xx', 'x@xx.x', 'xx@x.x', 'x@x.x.x', 'x@x.xxx', 'x@xx.xx', 'x@xxx.x', 'xx@x.xx', 'xx@xx.x', 'xxx@x.x', 'x@x.x.xx', 'x@x.xx.x', 'x@x.xxxx', 'x@xx.x.x', 'x@xx.xxx', 'x@xxx.xx', 'x@xxxx.x', 'xx@x.x.x', 'xx@x.xxx', 'xx@xx.xx', 'xx@xxx.x', 'xxx@x.xx', 'xxx@xx.x', 'xxxx@x.x']


# What's next?

If you like (and have time), define more test state machines in new test cells (test3, test4, etc). What types of string are easy to recognize, and which seem hard---or even impossible? You might want to try some of these and determine whether they're possible within the limits of state machines:

* Phone numbers
* URLs
* Palindromes
* HTML tags
* English-language numbers (e.g. "seventy-one", "nineteen", "six", "one hundred and one")
* Roman numerals

Another (bigger) project might be to write a regular expression compiler to generate state machines from regular expressions---even just supporting concatenation and Kleene star would be a fun exercise for an evening!

If you have an interest in the theory behind computer science, you might want to implement a non-deterministic automaton which can be in (or start in) several states simultaneously, may have out-edges to multiple distinct states on a given symbol, or may even have "null" or "epsilon" transitions which can be taken without consuming any input at all! Another good project here might involve taking the intersection or union of two languages (by manipulating their state machines).

In [99]:
#I was trying to make a generator for a StateMachine but I couldn't get it to work
'''
def edge_gen(re):
    result = []
    for i in range(1, len(re)):
        result.append(("s"+str(i-1),re[i-1],"s"+str(i)))
    return result

def sm_gen(re):
    return StateMachine(edge_gen(re),"s1",["s"+str(len(re))])

assert check(sm_gen('test'), 'test')
'''

#URLs
def test3():
    return StateMachine(
        [
            ("s1","h","s2"),("s2","t","s3"),("s3","t","s4"),("s4","p","s5"),
            ("s5","s","s5"),
            ("s5",":","s6"),("s6","/","s7"),("s7","/","s8"),("s8","w","s9"),("s9","w","s10"),("s10","w","s11"),("s11",".","s12"),
            #It works if I have a way to implement all characters here
            ("s12",".","s13"),
            #Here too
        ],
        "s1",
        #Terminal state depends on how many more states it would take
        [])

AssertionError: 